In [ ]:
# Pull MKTRF and RF
ff3 = pdr('F-F_Research_Data_Factors','famafrench', start=1900)[0]/100
est_mkt_std = ff3['Mkt-RF'].expanding(60).std()

x = raw_data.set_index(pd.to_datetime(raw_data.date, infer_datetime_format=True).dt.to_period('M'))
merged = pd.merge(x, ff3, how='left',left_index=True,right_index=True)
merged['xret'] = merged.ret - merged['RF']
cols = merged.columns
merged = merged.reset_index()
merged.columns = ['mdate'] +cols.to_list()
merged = merged.set_index(['mdate','ticker'])


In [ ]:
# Auxiliary functions
def estimate_betas(d,tickers,mdate):
    df_beta = pd.DataFrame(index=tickers, columns=['alpha','beta'],dtype=float)
    for ticker in tickers:
        subset = d.xs(ticker,level=1).loc[:mdate].iloc[:-1]
        y =subset.xret
        X = sm.add_constant(subset['Mkt-RF'])
        mm = sm.OLS(y, X, missing='drop').fit()
        df_beta.loc[ticker] = mm.params.values
    return df_beta['beta'].values

def is_pos_def(x):
    if np.all(np.linalg.eigvals(x) > 0):
        return 'True'
    else:
        return 'False'
    
def ls_constrained_gmv(n, cov, max_wgt):
    '''
    n: assets on each side of portfolio (n longs and n shorts; longs on top)
    '''
    Q = matrix(cov, tc="d")
    p = matrix(np.zeros(2*n), (2*n, 1), tc="d")
    # Constraint: (1) long-only for first n (2) short-only for remaining n (3) max long position = 0.05 (4) min short position = -0.05
    cons1 = np.hstack((-np.identity(n),np.zeros((n,n))))
    cons2 = np.hstack((np.zeros((n,n)),np.identity(n)))
    cons3 = -cons1
    cons4 = -cons2
    G = matrix(np.vstack((cons1,cons2,cons3,cons4)), tc="d")
    h = matrix(np.vstack((np.zeros(2*n),max_wgt*np.ones(2*n))).reshape(-1), (4*n, 1), tc="d")        
    # Constraint: (1) fully-invested portfolio (2) longs sum to 1.3 (3) shorts sum to -0.3
    cons1 = np.ones(2*n)
    cons2 = np.hstack((np.ones(n),np.zeros(n)))
    A = matrix(np.vstack((cons1,cons2)), (2, 2*n), tc="d")
    b = matrix([1, 1.3], (2, 1), tc="d")    
    sol = Solver(Q, p, G, h, A, b)
    wgts = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
    return wgts   
SolverOptions['show_progress'] = True 

In [ ]:
mdate_list = best.date.unique()
mdate_list

rets_gmv = pd.DataFrame(dtype=float,index=best_rets.index,columns = ['best','worst','ls'])

for MDATE in mdate_list:
    print(MDATE)

    # Extract securities selected that month
    b = best.set_index(['date','ticker']).xs(MDATE,level=0)
    w = worst.set_index(['date','ticker']).xs(MDATE,level=0)
    all= pd.concat([b,w])
    ticker_list = all.index.to_list()

    # Calculate expanding window betas for these tickers
    mkt_std = est_mkt_std.loc[:MDATE][-1]  # Pull last month's estimated market std dev
    betas = estimate_betas(merged,ticker_list,MDATE)
    betas = 0.67*betas+0.33               # Shrink betas towards 1
    sds_mkt_model = betas * mkt_std
    # SDS = all.lag_sd.fillna(value=all.lag_sd.mean()).values
    SDS = np.where(all.lag_sd.isnull(), 2*sds_mkt_model, all.lag_sd)
    COV = np.diag(SDS) @ np.identity(200) @ np.diag(SDS)
    for j, ticker in enumerate(ticker_list):
        for k in range(j+1,200):
            COV[j, k] = COV[k, j] = betas[j] * betas[k] * (mkt_std**2) 

    print(f'Positive definite?: {is_pos_def(COV)}')

    # Calculate constrained GMV portfolio and portfolio returns
    if is_pos_def(COV)=='True':
        wgts_gmv_cons = ls_constrained_gmv(100,COV,0.03)
    else:
        # Revert to equal-weighting if covariance matrix is not positive definite
        wgts_gmv_cons = np.hstack((1.3*np.ones(100)/100, -0.3*np.ones(100)/100))
    rets_gmv.loc[MDATE,'ls'] = all.ret @ wgts_gmv_cons
    rets_gmv.loc[MDATE,'best'] = all.ret[:100] @ wgts_gmv_cons[:100] / wgts_gmv_cons[:100].sum()
    rets_gmv.loc[MDATE,'worst'] = all.ret[100:] @ wgts_gmv_cons[100:] / wgts_gmv_cons[100:].sum()